# 五、生成

检索增强生成（Retrieval Augmented Generation，RAG）正成为大语言应用领域的热门解决方案。RAG 可以将大语言模型强大的理解能力和领域知识相结合，从而提高模型准确性和效率。RAG主要流程分为两步：1. 检索：从大量数据源中检索出和问题相关的内容；2.生成：将相关的知识拼接到prompt中，让LLM基于相关知识和用户问题进行回答。RAG可以更有效地利用垂域知识。


RAG中生成阶段发挥重要作用，其不仅要融合检索阶段的相关内容，还要利用先进自然语言处理技术来生成问题的回复。
以下是一个RAG Prompt的示例，帮助理解RAG中的生成阶段。

<img src="./rag_flow.png" width="680px">

上图中，假设已从外部数据库中检索出和用户query的相关内容，接下来将用户query、外部相关信息注入提示模板Prompt中，然后将prompt输入大模型LLM得到问题回复。

Prompt通常由任务描述、检索获得的背景知识，以及用户问题组成。

**结构化Prompt举例：** （商品客服问答场景）

[任务描述]

你是一个专业的客服机器人，请参考 [背景知识] 回答 [问题] 。

[背景知识]

{relevant information} // 检索阶段返回的相关信息

[问题]

{用户提问的 query} // eg. 有几种四件套？

设计Prompt后，便可将其输入大语言模型，得到生成结果。下面将代码案例说明如何使用。

In [1]:
#!/usr/bin/env python3
import os
import openai
# gpt 网关调用
os.environ["OPENAI_API_KEY"] = "{您的 api key}"
os.environ["OPENAI_API_BASE"] = "{您的 url}"
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

context = "1. 全棉床上四件套：材质：采用全棉材质，柔软舒适，对皮肤无刺激。设计：床单和被套的设计简约大方，色彩柔和，适合各种家居风格。尺寸：通常尺寸为203*229cm，适合双人使用，提供充足的睡眠空间；2. 萌趣布朗熊图案床上四件套：图案：萌趣的布朗熊图案增添童趣，让卧室焕发活力。材质：全棉面料，透气性好，柔软舒适；3. 学生宿舍专用床上四件套：适用性：适合学生宿舍单人或双人使用。材质与设计：纯棉材质，简约设计，易于搭配，打造舒适睡眠环境；4. 双面印花床上四件套：设计：A面采用可爱小动物和冰淇淋图案装饰，B面灵动波点，彰显少女甜美。材质：全棉面料，舒适柔软；5. 澳毛大豆纤维填充被芯套：填充物：采用51%澳毛和20%大豆纤维填充，保暖性能优良。设计：子母被设计，方便拆洗和更换，适合不同季节使用。"

prompt = ChatPromptTemplate.from_template(
    '''
        【任务描述】
        请根据用户输入的上下文回答问题，并遵守回答要求。

        【背景知识】
        {context}

        【回答要求】
        - 你需要根据背景知识的内容回答。
        - 对于不知道的信息，直接回答“未找到相关答案”
        -----------
        {question}
    '''
)

model = ChatOpenAI(model="gpt-35-turbo-1106")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"context":context, "question": "有几种四件套？"})

/Users/yinqing7/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'一共有五种四件套，分别是全棉床上四件套、萌趣布朗熊图案床上四件套、学生宿舍专用床上四件套、双面印花床上四件套和澳毛大豆纤维填充被芯套。'

## 5.1 后处理

后处理是RAG系统中的一个关键环节，通常发生在检索阶段之后，旨在优化和提升检索和生成结果的质量和相关性。

<img src="./postprocess.png" width="680px">

* Embedding Model : 将文档段落编码为向量的嵌入模型。
* Retriever：通过嵌入模型编码用户问题query，并返回嵌入问题附近的任意编码文档documents。
* 后处理（可选）: Compressor提取关键信息；ReRanker根据某规则重新计算query与documents间的得分。
* Language Model : 语言模型用于接收来自检索器或重排器的记录以及问题，并返回答案。

**为什么进行后处理？**
1. **优化信息密度**：由于LLM对输入的字数有限制，后处理可以通过筛选和压缩信息从大量检索结果中提炼出最核心的信息。这样可以确保即使在字数限制内，也能向模型提供最相关和最有价值的数据，从而生成高质量和高密度的信息内容。

2. **提高生成效率**：在有限的字数下，模型需要处理的信息越精炼，其生成答案的效率就越高。后处理通过去除冗余和不相关信息，确保模型专注于最关键的内容，这样可以在有限的交互中快速生成准确和有用的回答。

3. **确保内容的连贯性**：在字数受限的情况下，模型可能无法一次性接收并处理所有相关信息。后处理可以帮助组织和结构化信息，使其在生成阶段能够以连贯的方式呈现，避免因字数限制而导致的信息断层或不完整。

4. **提升生成文本的质量**：后处理不仅可以优化输入数据，还可以通过调整生成策略来提升输出文本的质量。例如，通过设置优先级，确保最重要的信息首先被包含在生成的文本中，即使在字数限制的情况下也能保证回答的核心价值。

后处理通常可包括以下方面：

1. **信息压缩**：由于检索阶段可能会返回大量相关信息，为了提高生成阶段的效率和性能，可对大量信息进行压缩，提取最关键的内容。这可能涉及到提取关键句子、短语或概念，以便在生成答案时能够集中于最相关的信息。

2. **重新排序**：根据与用户查询的相关性对检索结果进行重新排序。这通常基于文档与查询的匹配程度、文档的权威性、用户的历史偏好等因素。通过这种方式，最相关和最有用的信息会被放在最前面，以供生成模型优先考虑。

通过这些后检索处理步骤，RAG系统能够确保生成阶段的输入是精炼、相关且高质量的，从而提高最终生成文本的准确性和用户满意度。为了帮助理解上述后处理步骤，请看下面提供的案例。

[背景] ：用户对一款新型智能手机的功能和规格感兴趣，并提出了查询query。为了回答这个问题，RAG系统的检索阶段是从外部知识库中检索了相关的产品评测、技术规格表和用户评论。

[后处理阶段]
1. 信息压缩：由于检索到的文档数量庞大，首先对信息进行压缩即提取每篇文档中的关键句子和重要属性。如从技术规格表中提取了处理器型号、内存大小、屏幕分辨率等重要属性，且从用户评论中提取了关于电池续航、摄像头性能等的反馈。
2. 重新排序：根据用户查询的关键词和查询意图，对压缩后的信息进行了重新排序。如将用户评论中提到的电池续航问题放在前面，因为这是用户在购买决策中非常关心的一个方面。

[生成阶段]：系统将上述整合的上下文context输入到语言生成模型中，生成了一个涵盖了智能手机的主要功能、规格亮点以及使用反馈的回答。

### 5.1.1 信息压缩

**为什么进行信息压缩？**

信息压缩通常指的是减少数据量而不丢失关键信息的过程。以往的方法在整合LLMs到检索式问答框架中存在一定的局限性，如计算成本高、对长文本的处理不足等。为了提高生成阶段的效率和性能，可对检索阶段返回的大量信息进行压缩，提取最关键的内容，以便在生成答案时能够集中于最相关的信息。

**信息压缩**

信息压缩可以在RAG的retriever阶段对于返回的内容进行压缩即上下文压缩（contextual compression）；也可以对于已组成的结构化prompt信息进行压缩即提示压缩（prompt compression）。

- 上下文压缩：使用给定查询的上下文对文档进行压缩，从而只返回相关信息，而不是立即按原样返回检索到的文档。这里的 "压缩 "既指压缩单个文档的内容，也指整体过滤掉文档。可以借助langchain包中的[ContextualCompressionRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.contextual_compression.ContextualCompressionRetriever.html)实现此功能。
- 提示压缩：从粗到细的提示语压缩方法，可以在高压缩率下保持语义完整性的预算控制器。可以借助[LLMLingua](https://github.com/microsoft/LLMLingua)工具包，其利用一个紧凑、训练有素的语言模型（如 GPT2-small、LLaMA-7B）来识别和移除提示中的非必要标记。这种方法可实现大型语言模型 (LLM) 的高效推理，以最小的性能损失实现高达 20 倍的压缩。

1）**上下文压缩（Contextual Compression）案例**

In [2]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.document_loaders import TextLoader
from langchain.retrievers import ContextualCompressionRetriever

import os

os.environ["OPENAI_API_KEY"] = "{您的 api key}"
os.environ["OPENAI_API_BASE"] = "{您的 url}"


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 50}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )
    
    
loader = TextLoader('chinese_text.txt')
documents = loader.load()


text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=0)
split_documents = text_splitter.split_documents(documents)

retriever = FAISS.from_documents(split_documents, OpenAIEmbeddings()).as_retriever()

docs = retriever.invoke("中国的首都是哪里?")
pretty_print_docs(docs)


print('\n'+'**'*8+'\n')
print('过滤掉similarity score小于0.85的documents，compressor结果如下：\n')


embeddings = OpenAIEmbeddings()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.85)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "中国的首都是哪里?"
)
pretty_print_docs(compressed_docs)

Document 1:

北京,是中华人民共和国的首都,也是中国的政治、文化、科技、教育和国际交往中心。北京位于华北平原的东北边缘,背靠燕山,毗邻天津市和河北省。北京是一座有着三千多年建城史和八百多年建都史的历史文化名城。

北京的历史可以追溯到西周时期。公元前1045年,周武王封召公奭于燕国,建城于蓟,这是北京建城的开始。自金、元、明、清以来,北京一直是中国的首都,见证了中国历史的沧桑巨变。
--------------------------------------------------
Document 2:

北京拥有众多名胜古迹,其中最著名的要数故宫博物院。故宫是明清两朝的皇宫,是世界上现存规模最大、保存最完整的木质结构古建筑群。天安门广场是北京的象征,每天都有大量中外游客来到这里参观。长城、颐和园、天坛、北海公园等都是北京的标志性景点。

作为中国的文化中心,北京汇聚了众多的博物馆、图书馆和艺术馆。故宫博物院、中国国家博物馆、首都博物馆等都藏有大量珍贵的文物和艺术品。国家图书馆是中国最大的图书馆,藏书量超过3000万册。国家大剧院、长安大戏院等演出场所常年上演精彩的文艺节目。
--------------------------------------------------
Document 3:

北京也是中国的科技和教育中心。中关村被誉为"中国硅谷",聚集了大量高科技企业和研发机构。北京大学、清华大学、中国人民大学等著名高校坐落于北京,吸引了来自全国各地和世界各国的学子前来求学。

近年来,北京的交通基础设施建设取得长足进步。北京首都国际机场是中国最大的航空港,与世界各地通航。北京地铁路网四通八达,连接了城市的各个角落。京张高铁的开通,使得北京与张家口两地时间距离大大缩短。
--------------------------------------------------
Document 4:

2008年,北京成功举办了第29届夏季奥运会,向世界展示了一个古老而现代、包容而开放的中国首都形象。2022年,北京又成为冬奥会的主办城市,成为全球首个"双奥之城"。

如今的北京,正朝着国际一流的和谐宜居之都的目标迈进。在建设国际一流的和谐宜居之都的过程中,北京将继续发挥首都的示范引领作用,为实现中华民族伟大复兴的中国梦贡献首都力量。

*****

**上述代码执行结果表明，通过限制返回的文本块与用户query间的相关性，可以过滤掉 Document 2、3和4，从而实现上下文压缩。**

2）**提示压缩（Prompt Compression）案例**

In [3]:
from llmlingua import PromptCompressor

question = "回答关于大模型的相关信息"
context =  [
    "Prompt【可选】◦告知LLM内system服从什么角色◦占位符：设置{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG（Retrieval-Augmented Generation），RAG 为了解决LLM中语料的通用和时间问题，通过增加最新的或者垂类场景下的外部语料，Embedding化后存入向量数据库，然后模型从外部语料中寻找相似语料辅助回复•Models◦可做 Embedding化，语句补全，对话等支持的模型选择，OpenAI为例•Parser【可选】◦StringParser，JsonParser 等◦将模型输出的AIMessage转化为string, json等易读格式上述介绍了Langchain开发中常见的components，接下来将通过一简单案例将上述组件串起来，让大家更熟悉Langchain中的组件及接口调用。",
    "LangChain 作为一个大语言模型（LLM）集成框架，旨在简化使用大语言模型的开发过程，包括如下组件： LangChain框架优点：1.多模型支持：LangChain 支持多种流行的预训练语言模型，如 OpenAI GPT-3、Hugging Face Transformers 等，为用户提供了广泛的选择。2.易于集成：LangChain 提供了简单直观的API，可以轻松集成到现有的项目和工作流中，无需深入了解底层模型细节。3.强大的工具和组件：LangChain 内置了多种工具和组件，如文档加载器、文本转换器、提示词模板等，帮助开发者处理复杂的语言任务。4.可扩展性：LangChain 允许开发者通过自定义工具和组件来扩展框架的功能，以适应特定的应用需求。5.性能优化：LangChain 考虑了性能优化，支持高效地处理大量数据和请求，适合构建高性能的语言处理应用。6.Python 和 Node.js 支持：开发者可以使用这两种流行的编程语言来构建和部署LangChain应用程序。由于支持 Node.js ，前端大佬们可使用Javascript语言编程从而快速利用大模型能力，无需了解底层大模型细节。同时也支持JAVA开发，后端大佬同样适用。本篇文章案例聚焦Python语言开发。",
    "LangChain表达式 (LCEL)LangChain表达式语言，或者LCEL，是一种声明式的方式，可以轻松地将链条组合在一起。 LCEL从第一天开始就被设计为支持将原型放入生产中，不需要改变任何代码，从最简单的“提示+LLM”链到最复杂的链(我们已经看到人们成功地在生产中运行了包含数百步的LCEL链)。以下是你可能想要使用LCEL的一些原因：流式支持 当你用LCEL构建你的链时，你可以得到最佳的首次到令牌的时间(输出的第一块内容出来之前的时间)。对于一些链，这意味着例如我们直接从LLM流式传输令牌到一个流式输出解析器，你可以以与LLM提供者输出原始令牌相同的速率得到解析后的、增量的输出块。异步支持 任何用LCEL构建的链都可以通过同步API(例如在你的Jupyter笔记本中进行原型设计时)以及异步API(例如在LangServe服务器中)进行调用。这使得可以使用相同的代码进行原型设计和生产，具有很好的性能，并且能够在同一台服务器中处理许多并发请求。优化的并行执行 无论何时，你的LCEL链有可以并行执行的步骤(例如，如果你从多个检索器中获取文档)，我们都会自动执行，无论是在同步接口还是异步接口中，以获得最小可能的延迟。重试和回退 为你的LCEL链的任何部分配置重试和回退。这是一种使你的链在大规模下更可靠的好方法。我们目前正在努力为重试/回退添加流式支持，这样你就可以在没有任何延迟成本的情况下获得增加的可靠性。访问中间结果 对于更复杂的链，通常在最终输出产生之前就能访问中间步骤的结果是非常有用的。这可以用来让最终用户知道正在发生什么，甚至只是用来调试你的链。你可以流式传输中间结果，它在每个LangServe服务器上都可用。输入和输出模式 输入和输出模式为每个LCEL链提供了从你的链的结构中推断出来的Pydantic和JSONSchema模式。这可以用于验证输入和输出，是LangServe的一个重要部分。无缝的LangSmith跟踪集成 随着你的链变得越来越复杂，理解在每一步究竟发生了什么变得越来越重要。 使用LCEL，所有步骤都会自动记录到LangSmith，以实现最大的可观察性和可调试性。"
]

llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
    use_llmlingua2=True,
    device_map="cpu",
)

compressed_prompt = llm_lingua.compress_prompt(
    context,
    question=question,
    # Set the special parameter for LongLLMLingua
    condition_in_question="after_condition",
    reorder_context="sort",
    dynamic_context_compression_ratio=0.3, 
    condition_compare=True,
    context_budget="+100",
    rank_method="longllmlingua",
)
compressed_prompt

Token indices sequence length is longer than the specified maximum sequence length for this model (547 > 512). Running this sequence through the model will result in indexing errors


{'compressed_prompt': 'Prompt【可选】◦告知LLM内system服从什么角色◦占位符{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG(Retrieval-Augmented,Embedding化后存入向量数据库 Embedding化,语句补全,OpenAI为例•Parser【可选】◦StringParser,JsonParser json等易读格式上述介绍了Langchain开发中常见的components\n\nLangChain 作为一个大语言模型)集成框架 LangChain框架优点:1.多模型支持 支持多种流行的预训练语言模型 OpenAI GPT-3、Hugging Face Transformers 提供了简单直观的API.强大的工具和组件,如文档加载器、文本转换器、提示词模板等.可扩展性.性能优化,适合构建高性能的语言处理应用。6.Python 和 Node.js 支持 Node.js,前端大佬们可使用Javascript语言编程从而快速利用大模型能力,后端大佬同样适用。本篇文章案例聚焦Python语言开发。',
 'compressed_prompt_list': ['Prompt【可选】◦告知LLM内system服从什么角色◦占位符{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG(Retrieval-Augmented,Embedding化后存入向量数据库 Embedding化,语句补全,OpenAI为例•Parser【可选】◦StringParser,JsonParser json等易读格式上述介绍了Langchain开发中常见的components',
  'LangChain 作为一个大语言模型)集成框架 LangChain框架优点:1.多模型支持 支持多种流行的预训练语言模型 OpenAI GPT-3、Hugging Face Transformers 提供了简单直观的API.强大的工具和组件,如文档加载器、文本转换器、提示词模板等.可扩展性.性能优化,适合构建高性能的语言处理应用。6.Python 和 Node.js 支持 Node.js,前端大佬们可使用Javascript语言编程从

**将上下文和用户query整合为提示prompt后，利用提示压缩减少整体字符数，从而做到减少调用大模型接口成本。**

### 5.1.2 重新排序

1. **什么是重排？**

<img src="reranker.png" width="680px">

重新排序模型（也称为交叉编码器）是在给定查询和文档对的情况下，它将输出一个相似性得分。利用这个分数，按照与查询的相关性对文档重新排序。

长期以来，搜索工程师通常使用两阶段检索系统（检索器+重排序）。其中，第一阶段模型（嵌入模型/检索器）从一个更大的数据集中检索一组相关文档。然后，使用第二阶段模型（重排序器）对第一阶段模型检索到的文档进行重排序。

使用两个阶段是因为，从大型数据集中检索一小部分文档要比重新排序一大部分文档快得多--简单来说，重排序器速度慢，而检索器速度快。


2. **为什么要重排？**


<img src="reranker2.png" width="680px">

尽管嵌入向量按余弦相似度排序，但这并不能保证最相关的内容会排在最前面。这部分是因为矢量搜索通常依赖于预先计算的嵌入向量，这些向量可能无法完全捕捉到查询特定的相关性（即在用户查询之前就创建了嵌入向量，没有考虑查询的上下文）。

这些限制凸显了需要一种方法来根据其与手头特定查询的相关性进一步细化和重新排序检索结果。这正是更复杂的排名技术发挥作用的地方。

为了克服矢量搜索的限制，可以将重排序器整合到RAG流程中，作为数据过滤。该重排序器基于交叉编码器，能准确度量文本相似度，并通过特定排名损失函数训练优化。它输出的分数基于直接文本比较，比向量语义相似度更精确，同时考虑了语义、上下文和微妙含义。

重排序器虽然更准确，但速度明显慢于向量相似度计算。因此，重排序器通常在计算出向量相似度之后作为第二阶段使用。这种两阶段方法结合了矢量搜索的速度和重排序器的精确度。

因此采用两阶段的检索（嵌入模型/检索器，重排器）做到时间和精度上的权衡。

3. **重排有什么选择？**

下图中，对比了加入重排 vs. 无重排 (WithoutReranker) 的效果，其中加入重排的 bge-reranker-base、bge-reranker-large和cohere-reranker在准确度上表现更好，表明重排有助于优化检索到的文档的顺序，从而提高生成答案的准确性和相关性。

<img src="reranker_perform.png" width="680px">

重排可有如下选择：

* 交叉编码器（cross encoder）【免费】：模型通过采用数据对的分类机制重新定义了传统方法。该模型以一对数据，例如两个句子作为输入，并产生一个介于0和1之间的输出值，表示两个项目之间的相似性。这种与向量嵌入的背离允许对数据点之间的关系有更微妙的理解。需要注意的是，交叉编码器需要为每个输入提供一对“项目”，这使它们不适合独立处理单个句子。在搜索的上下文中，交叉编码器与每个数据项和搜索查询一起使用，以计算查询与数据对象之间的相似性。开源的[BAAI/bge-reranker](https://huggingface.co/BAAI/bge-reranker-base)模型（如 bge-reranker-base、bge-reranker-large、和 bge-reranker-large-en-v1.5等）属于这一类。

* 重排API【收费】：Cohere提供的[排序模型](https://docs.cohere.com/docs/rerank-2) （如 rerank-multilingual-v3.0 等）;  [Jina](https://jina.ai/reranker/)


4. **案例**


1） **重新排序模型 bge-reranker-large**

bge-reranker-large 是一个基于 BERT 的重排模型,用于对候选文档进行重新排序,以提高检索结果的相关性。
与嵌入模型embedding不同，reranker 以问题和文档为输入，直接输出相似度而非嵌入。为了兼顾准确性和时间成本，交叉编码器被广泛用于对其他简单模型检索到的 top-k 文档进行重新排序。

In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

pairs = [['今天天气真如何?', '苹果很甜。'], ['今天天气如何?', '阴天，飘雨。']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([-9.3982,  0.6333])


`tensor([-9.3982,  0.6333])` 代表['今天天气如何?', '阴天，飘雨。']的相关度更高。

2） **进阶：考虑多样性的MMR重排**

a) *MMR是什么？*

Maximal Marginal Relevance (MMR) 是一种用于选择信息子集的准则，特别是在信息检索和机器学习领域中，用于生成摘要或推荐系统。MMR 旨在在保持高相关性的同时最大化多样性，确保所选信息的覆盖面广泛且不重复。

MMR 的核心思想是通过以下两个标准来选择信息：

1. **边际相关性**（Marginal Relevance）：选择与用户查询或目标任务最相关的信息。通常通过计算信息与查询之间的相似度或相关性得分来实现。

2. **边际多样性**（Marginal Diversity）：在选择信息时，考虑新信息与已选择信息集合之间的差异。这有助于避免选择过于相似的信息，从而增加信息集合的多样性。

MMR 的目标是在满足用户查询的相关性要求的同时，提供尽可能多的不同视角或信息点。这通常通过以下步骤实现：

- 计算每个候选信息与用户查询的相关性得分。
- 选择与查询最相关的信息作为初始集合。
- 对于接下来的每个选择，计算候选信息与查询的相关性得分，同时考虑其与已选择信息集合的多样性得分。
- 选择在相关性和多样性上得分最高的信息加入集合。

MMR 方法的一个关键优势是它能够在保持高相关性的同时，提供多样化的信息，这对于生成摘要、构建推荐列表或进行多文档摘要等任务非常有用。然而，MMR 也需要仔细平衡相关性和多样性，以避免过度强调其中之一而损害整体效果。

b) *如何在RAG中使用MMR ReRanker？*

In [12]:
import os

# gpt 网关调用
os.environ["OPENAI_API_KEY"] = "{您的 api key}"
os.environ["OPENAI_API_BASE"] = "{您的 url}"

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-35-turbo-1106", temperature=0.2)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# load data and index
documents = SimpleDirectoryReader("./data/paul_graham1/").load_data()
index = VectorStoreIndex.from_documents(documents)

# common retriever
retriever = index.as_retriever(
    similarity_top_k=3
)
nodes = retriever.retrieve(
    "作者在Y Combinator工作期间都做了些什么?"
)

from llama_index.core.response.notebook_utils import display_source_node

for n in nodes:
    display_source_node(n, source_length=200)

    
print('*'*18)

# with mmr reranker

mmr_retriever = index.as_retriever(
    vector_store_query_mode="mmr",
    similarity_top_k=3,
    vector_store_kwargs={"mmr_threshold": 0.5},
)
mmr_nodes = mmr_retriever.retrieve(
    "作者在Y Combinator工作期间都做了些什么?"
)

for n in mmr_nodes:
    display_source_node(n, source_length=200)

**Node ID:** c5b9d063-930d-4201-b883-b2d4333d5e83<br>**Similarity:** 0.8663631081758175<br>**Text:** 我认为第一批如此出色并不完全是运气。你必须非常大胆才能报名参加像夏季创始人计划这样奇怪的事情，而不是在像微软或高盛这样合法的地方找到暑期工作。

初创公司的交易是基于我们与Julian达成的交易（1万美元换10%）和Robert所说的麻省理工学院研究生夏季获得的交易（6千美元）的结合。我们投资了每位创始人6千美元，在典型的两位创始人的情况下是1.2万美元，作为回报，我们获得了6%的股份。这一...<br>

**Node ID:** a1cd6291-2faf-4646-80fa-3a5e17351f87<br>**Similarity:** 0.8622093553874532<br>**Text:** 这反过来意味着，在快速变化影响的领域（习俗更有可能过时的地方），独立思考的人（即受习俗影响较小的人）将具有优势。

不过，这里有一个有趣的点：你并不总能预测哪些领域会受到快速变化的影响。显然，软件和风险投资会，但谁会预料到论文写作会呢？

[13] Y Combinator 并不是最初的名字。起初我们被称为 Cambridge Seed。但我们不想要一个地区性的名字，以防有人在硅谷复制我们，...<br>

**Node ID:** ef788629-eeb3-4dd1-9413-a4721832b67a<br>**Similarity:** 0.8600962311321759<br>**Text:** 但演讲后，我意识到我确实应该停止拖延天使投资。自从Yahoo收购了我们，我就一直在考虑这个问题，现在已经过去7年了，我还没有做过一次天使投资。

与此同时，我一直在和Robert和Trevor策划我们可以一起工作的项目。我错过了和他们一起工作的日子，似乎肯定有我们可以合作的事情。

当我们在3月11日的晚餐后走在Garden和Walker街的拐角处时，这三个线索汇聚在一起。去他的那些VC，他...<br>

******************


**Node ID:** c5b9d063-930d-4201-b883-b2d4333d5e83<br>**Similarity:** 0.4331141773767519<br>**Text:** 我认为第一批如此出色并不完全是运气。你必须非常大胆才能报名参加像夏季创始人计划这样奇怪的事情，而不是在像微软或高盛这样合法的地方找到暑期工作。

初创公司的交易是基于我们与Julian达成的交易（1万美元换10%）和Robert所说的麻省理工学院研究生夏季获得的交易（6千美元）的结合。我们投资了每位创始人6千美元，在典型的两位创始人的情况下是1.2万美元，作为回报，我们获得了6%的股份。这一...<br>

**Node ID:** a1cd6291-2faf-4646-80fa-3a5e17351f87<br>**Similarity:** -0.027827916071308745<br>**Text:** 这反过来意味着，在快速变化影响的领域（习俗更有可能过时的地方），独立思考的人（即受习俗影响较小的人）将具有优势。

不过，这里有一个有趣的点：你并不总能预测哪些领域会受到快速变化的影响。显然，软件和风险投资会，但谁会预料到论文写作会呢？

[13] Y Combinator 并不是最初的名字。起初我们被称为 Cambridge Seed。但我们不想要一个地区性的名字，以防有人在硅谷复制我们，...<br>

**Node ID:** b0c8a1fe-870e-4334-8b4d-755a78a567a1<br>**Similarity:** -0.023409849601093047<br>**Text:** ”然而，这再次不是我们的特别洞察。我们不知道风险投资公司是如何组织的。我们从未考虑过尝试筹集一个基金，如果这样做，我们也不知道从哪里开始。[14]

YC最独特的一点是批量模型：一次资助一群初创公司，每年两次，然后花三个月的时间集中精力帮助它们。这部分我们是偶然发现的，不仅仅是隐含地，而是由于我们对投资的无知而明确地发现的。我们需要作为投资者的经验。有什么比一次资助一大堆初创公司更好的方法呢...<br>

**上述代码执行结果表明，普通Retriever返回的TOP3文档相关性均约为0.86，而加入MMR ReRanker的Retriever返回的TOP3文档不仅考虑了相关性，也考虑了多样性，体现在返回的第三个文档对比不同，增加了内容的多样性。**

## 5.2 RAG&Fine-tuning

1. *RAG vs. Fine-tuning*

<img src="./rag&ft.png" width="300px">

* 动态数据
    * RAG : 直接更新检索知识库，无需频繁重新训练，适合动态变化的数据场景。
    * Fine-tuning : 存储静态数据，需要重新训练用于知识更新。
    
* 可解释性
    * RAG : 回复能追溯到具体数据来源，提供更高的可解释性和可追踪性。
    * Fine-tuning : 黑盒模型，不总能清楚模型为何做出此反应。

* 降低幻觉
    * RAG : 生成的回复基于检索到的实际内容，不易产生虚构。
    * Fine-tuning : 根据特定领域数据训练有助于减少幻觉，但在未训练过的输入上仍可能出现幻觉。
    
* 模型定制
    * RAG : 侧重于信息检索和融合外部知识，但无法充分定制模型行为。
    * Fine-tuning : 允许根据特定数据调整LLM行为、写作风格和领域知识。
    
2. *使用场景举例*

<img src="raft.png" width="880px">

其中 open-book代表RAG，closed book代表Fine-tuning, RAFT代表RAG+Fine-tuning。

介绍了上述RAG和Fine-tuning的区别，下面根据实际场景介绍上述方法的使用。Remark: RAG和Fine-tuning非互斥，可搭配使用。

* 总结(Summarization)
    * 需要特定的领域和写作风格，选择Fine-tuning

* 问题解答(Question answering) 
    * 针对公司内部文件的问题解答系统
    * 需要动态更新数据，选择RAG

* 客户支持聊天机器人(Customer support chatbot)
    * 回答电子商务网站的问题
    * 需要动态更新数据
    * 训练特定语调和行为
    * RAG + Fine-tuning

* 代码生成(Code Generation)
    * 基于私有和公共代码库的代码建议系统
    * 需要按照代码规范生成
    * RAG + Fine-tuning
    
大家可以根据具体的使用场景，选择上述两种方法或者组合使用。


## 5.3 参考引用

**为什么要采用参考引用？**
参考引用指示生成的内容来自于检索的哪些文档。

1. 提供可解释性:通过参考引用,可以明确知道生成的答案中的不同部分来自于哪些源文档。这提供了一种可解释性,让用户或开发者能够追踪答案的来源,并评估答案的可靠性。

2. 增强答案的准确性:参考引用可以帮助生成模型更准确地生成答案。通过引用相关文档中的内容,模型可以更好地利用检索到的知识来回答问题,减少生成不相关或错误信息的可能性。

3. 促进知识的整合:参考引用允许模型将来自不同文档的信息进行整合,生成更全面和连贯的答案。通过引用多个相关文档,模型可以综合不同来源的知识,生成更加完善的答案。

4. 便于答案的后处理:参考引用提供了一种结构化的方式来表示答案中的知识来源。这种结构化信息可以方便地进行后处理,如将文档ID替换为实际的文档标题、链接或摘要等,使答案更加人性化和易于理解。

5. 支持答案的评估和改进:通过参考引用,可以方便地评估生成答案的质量。可以比较生成的答案与引用文档的相关性和一致性,从而识别模型生成答案的优缺点。这有助于发现模型的局限性,并为进一步改进模型提供指导。

下面将通过案例帮助大家理解 _参考引用_ 及其实现，面对多个文档，如何分chunks方便索引。

1） RAG的外部知识库包含多个文档，分割chunk，从chunks中寻找和用户提问相近的chunks，并输出chunk对应的文档链接及引用内容。

In [1]:
import os
# gpt 网关调用
os.environ["OPENAI_API_KEY"] = "{您的 api key}"
os.environ["OPENAI_API_BASE"] = "{您的 url}"

import openai
openai.api_key = os.environ['OPENAI_API_KEY']

from llama_index.readers.file import UnstructuredReader
from pathlib import Path
from llama_index.llms.openai import OpenAI
from llama_index.core import Document

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.storage.docstore import SimpleDocumentStore

from llama_index.core.schema import IndexNode
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SummaryIndex
from llama_index.core.retrievers import RecursiveRetriever
import os
from tqdm.notebook import tqdm
import pickle


reader = UnstructuredReader()
    
all_txt_files = [
    "data/background/component.txt",
    "data/background/intro.txt",
    "data/background/lcel.txt"
]

txt_contents = [
    "Prompt【可选】◦告知LLM内system服从什么角色◦占位符：设置{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG（Retrieval-Augmented Generation），RAG 为了解决LLM中语料的通用和时间问题，通过增加最新的或者垂类场景下的外部语料，Embedding化后存入向量数据库，然后模型从外部语料中寻找相似语料辅助回复•Models◦可做 Embedding化，语句补全，对话等支持的模型选择，OpenAI为例•Parser【可选】◦StringParser，JsonParser 等◦将模型输出的AIMessage转化为string, json等易读格式上述介绍了Langchain开发中常见的components，接下来将通过一简单案例将上述组件串起来，让大家更熟悉Langchain中的组件及接口调用。",
    "LangChain 作为一个大语言模型（LLM）集成框架，旨在简化使用大语言模型的开发过程，包括如下组件： LangChain框架优点：1.多模型支持：LangChain 支持多种流行的预训练语言模型，如 OpenAI GPT-3、Hugging Face Transformers 等，为用户提供了广泛的选择。2.易于集成：LangChain 提供了简单直观的API，可以轻松集成到现有的项目和工作流中，无需深入了解底层模型细节。3.强大的工具和组件：LangChain 内置了多种工具和组件，如文档加载器、文本转换器、提示词模板等，帮助开发者处理复杂的语言任务。4.可扩展性：LangChain 允许开发者通过自定义工具和组件来扩展框架的功能，以适应特定的应用需求。5.性能优化：LangChain 考虑了性能优化，支持高效地处理大量数据和请求，适合构建高性能的语言处理应用。6.Python 和 Node.js 支持：开发者可以使用这两种流行的编程语言来构建和部署LangChain应用程序。由于支持 Node.js ，前端大佬们可使用Javascript语言编程从而快速利用大模型能力，无需了解底层大模型细节。同时也支持JAVA开发，后端大佬同样适用。本篇文章案例聚焦Python语言开发。",
    "LangChain表达式 (LCEL)LangChain表达式语言，或者LCEL，是一种声明式的方式，可以轻松地将链条组合在一起。 LCEL从第一天开始就被设计为支持将原型放入生产中，不需要改变任何代码，从最简单的“提示+LLM”链到最复杂的链(我们已经看到人们成功地在生产中运行了包含数百步的LCEL链)。以下是你可能想要使用LCEL的一些原因：流式支持 当你用LCEL构建你的链时，你可以得到最佳的首次到令牌的时间(输出的第一块内容出来之前的时间)。对于一些链，这意味着例如我们直接从LLM流式传输令牌到一个流式输出解析器，你可以以与LLM提供者输出原始令牌相同的速率得到解析后的、增量的输出块。异步支持 任何用LCEL构建的链都可以通过同步API(例如在你的Jupyter笔记本中进行原型设计时)以及异步API(例如在LangServe服务器中)进行调用。这使得可以使用相同的代码进行原型设计和生产，具有很好的性能，并且能够在同一台服务器中处理许多并发请求。优化的并行执行 无论何时，你的LCEL链有可以并行执行的步骤(例如，如果你从多个检索器中获取文档)，我们都会自动执行，无论是在同步接口还是异步接口中，以获得最小可能的延迟。重试和回退 为你的LCEL链的任何部分配置重试和回退。这是一种使你的链在大规模下更可靠的好方法。我们目前正在努力为重试/回退添加流式支持，这样你就可以在没有任何延迟成本的情况下获得增加的可靠性。访问中间结果 对于更复杂的链，通常在最终输出产生之前就能访问中间步骤的结果是非常有用的。这可以用来让最终用户知道正在发生什么，甚至只是用来调试你的链。你可以流式传输中间结果，它在每个LangServe服务器上都可用。输入和输出模式 输入和输出模式为每个LCEL链提供了从你的链的结构中推断出来的Pydantic和JSONSchema模式。这可以用于验证输入和输出，是LangServe的一个重要部分。无缝的LangSmith跟踪集成 随着你的链变得越来越复杂，理解在每一步究竟发生了什么变得越来越重要。 使用LCEL，所有步骤都会自动记录到LangSmith，以实现最大的可观察性和可调试性。无缝的LangServe部署集成 任何用LCEL创建的链都可以使用LangServe轻松部署。"
]

doc_limit = 10

docs = []
for idx, f in enumerate(all_txt_files):
    if idx > doc_limit:
        break
    print(f"Idx {idx}/{len(all_txt_files)}")
    loaded_doc = Document(
        id_=str(f),
        text=txt_contents[idx],
        metadata={"path": str(f)},
    )
    print(str(f))
    docs.append(loaded_doc)

# pip install unstructured

embed_model = OpenAIEmbedding(
    model_name="text-embedding-ada-002", api_key=os.environ["OPENAI_API_KEY"]
)

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")

for doc in docs:
    embedding = embed_model.get_text_embedding(doc.get_content())
    doc.embedding = embedding

docstore = SimpleDocumentStore()
docstore.add_documents(docs)

# 构建索引
def build_index(docs, out_path: str = "storage/chunk_index"):
    nodes = []

    splitter = SentenceSplitter(chunk_size=512, chunk_overlap=70)
    for idx, doc in enumerate(tqdm(docs)):

        cur_nodes = splitter.get_nodes_from_documents([doc])
        for cur_node in cur_nodes:
            # ID will be base + parent
            file_path = doc.metadata["path"]
            new_node = IndexNode(
                text=cur_node.text or "None",
                index_id=str(file_path),
                metadata=doc.metadata
            )
            nodes.append(new_node)
    print("num nodes: " + str(len(nodes)))

    # save index to disk
    if not os.path.exists(out_path):
        index = VectorStoreIndex(nodes, embed_model=embed_model)
        index.set_index_id("simple_index")
        index.storage_context.persist(f"./{out_path}")
    else:
        # rebuild storage context
        storage_context = StorageContext.from_defaults(
            persist_dir=f"./{out_path}"
        )
        # load index
        index = load_index_from_storage(
            storage_context, index_id="simple_index", embed_model=embed_model
        )

    return index

index = build_index(docs)

out_top_k = 3

base_retriever = index.as_retriever(similarity_top_k=out_top_k)

# 用于展示文本块来源
def show_nodes(nodes, out_len: int = 512):
    for idx, n in enumerate(nodes):
        print(f"\n\n >>>>>>>>>>>> ID {n.id_}: {n.metadata['path']}")
        print(n.get_content()[:out_len])
        
query_str = "关于大模型的信息"

base_nodes = base_retriever.retrieve(query_str)

show_nodes(base_nodes)

Idx 0/3
data/background/component.txt
Idx 1/3
data/background/intro.txt
Idx 2/3
data/background/lcel.txt


  0%|          | 0/3 [00:00<?, ?it/s]

num nodes: 4


 >>>>>>>>>>>> ID 4cb3f47c-4290-4344-9f1f-afd49bb24227: data/background/intro.txt
LangChain 作为一个大语言模型（LLM）集成框架，旨在简化使用大语言模型的开发过程，包括如下组件： LangChain框架优点：1.多模型支持：LangChain 支持多种流行的预训练语言模型，如 OpenAI GPT-3、Hugging Face Transformers 等，为用户提供了广泛的选择。2.易于集成：LangChain 提供了简单直观的API，可以轻松集成到现有的项目和工作流中，无需深入了解底层模型细节。3.强大的工具和组件：LangChain 内置了多种工具和组件，如文档加载器、文本转换器、提示词模板等，帮助开发者处理复杂的语言任务。4.可扩展性：LangChain 允许开发者通过自定义工具和组件来扩展框架的功能，以适应特定的应用需求。5.性能优化：LangChain 考虑了性能优化，支持高效地处理大量数据和请求，适合构建高性能的语言处理应用。6.Python 和 Node.js 支持：开发者可以使用这两种流行的编程语言来构建和部署LangChain应用程序。由于支持 Node.js ，前端大佬们可使用Javascript语言编程从而快速利用大模型能力，无需了解底层大模型细节。同时也


 >>>>>>>>>>>> ID 8511f7fd-c570-48ba-87bb-bdfe764d70f1: data/background/component.txt
Prompt【可选】◦告知LLM内system服从什么角色◦占位符：设置{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG（Retrieval-Augmented Generation），RAG 为了解决LLM中语料的通用和时间问题，通过增加最新的或者垂类场景下的外部语料，Embedding化后存入向量数据库，然后模型从外部语料中寻找相似语料辅助回复•Models◦可做 Embedding化，语句补全，对话等支持的模型选择，OpenAI为例•Parser【可选】◦StringParser，JsonParser 等◦将模型输出的AIMessa

**上述代码执行结果 ID XXX: YYY 中 XXX代表文本块chunk id，YYY代表来自于哪个文档。帮助我们理解大模型生成结果参考了哪些引用。**

**参考链接与文献** 
1. https://www.pinecone.io/learn/series/rag/rerankers/ 
2. https://towardsdatascience.com/improving-rag-performance-using-rerankers-6adda61b966d 
3. https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83 
4. Zhang T, Patil S G, Jain N, et al. Raft: Adapting language model to domain specific rag[J]. arXiv preprint arXiv:2403.10131, 2024. 
